<a href="https://colab.research.google.com/github/Banugopan/Build_AI/blob/main/git_chatbot_ai_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

🔹 Block 0 – Installation et Ngrok

In [ ]:
!pip -q install fastapi uvicorn twilio nest_asyncio pyngrok openai

from pyngrok import ngrok, conf
ngrok.kill()  # ferme tous les anciens tunnels (évite erreurs 502)

# === authtoken ngrok ===
NGROK_AUTHTOKEN = ""
ngrok.set_auth_token(NGROK_AUTHTOKEN)
conf.get_default().region = "eu"  # proche de Paris

public_tunnel = ngrok.connect(8000, "http")
PUBLIC_BASE_URL = public_tunnel.public_url
print("PUBLIC_BASE_URL =", PUBLIC_BASE_URL)


PUBLIC_BASE_URL = https://unputridly-roseolar-sharlene.ngrok-free.dev


🔹 Block 1 – Variables Twilio et OpenAI

In [ ]:
import os

# === Twilio ===
TWILIO_ACCOUNT_SID = ""
TWILIO_AUTH_TOKEN  = ""
TWILIO_FROM        = "+"  # numéro Twilio acheté (E.164 : +44, +1, +33...)

# === OpenAI ===
os.environ["OPENAI_API_KEY"] = ""  # ta clé OpenAI

🔹 Block 2 – Application FastAPI

In [ ]:
# === Block 2 : Application FastAPI (FR) ===
from fastapi import FastAPI, Request
from fastapi.responses import Response
from twilio.rest import Client
import nest_asyncio, uvicorn, threading, time, os
from openai import OpenAI

nest_asyncio.apply()

# --- Clients ---
twilio_client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)
client_ai = OpenAI()                                # lit OPENAI_API_KEY
client_ai_fast = client_ai.with_options(timeout=20) # timeout GPT (20s)

# --- FastAPI app ---
app = FastAPI()

def twiml(xml: str) -> Response:
    """Retourne une réponse TwiML (XML) pour Twilio."""
    return Response(content=xml, media_type="text/xml; charset=utf-8")

@app.get("/health")
def health():
    return {"ok": True, "base": PUBLIC_BASE_URL}

@app.post("/call")
async def make_call(to: str):
    """Déclenche un appel sortant via Twilio vers le numéro 'to'."""
    print(f">>> /call -> {to}", flush=True)
    call = twilio_client.calls.create(
        to=to,
        from_=TWILIO_FROM,
        url=f"{PUBLIC_BASE_URL}/voice"   # Twilio ira chercher le TwiML ici
        # method="GET"  # décommente si tu veux forcer GET
    )
    print(f">>> CALL SID = {call.sid}", flush=True)
    return {"sid": call.sid}

# ✅ Accepte GET et POST (selon config Twilio)
@app.api_route("/voice", methods=["GET", "POST"])
async def voice():
    """Première réponse vocale envoyée à Twilio (invite l’utilisateur à parler/appuyer)."""
    print(">>> /voice (GET/POST)", flush=True)
    xml = f"""<?xml version="1.0" encoding="UTF-8"?>
<Response>
  <Say language="fr-FR" voice="alice">Bonjour ! Je suis votre manageur , vous allez bien ? .</Say>
  <Pause length="1"/>
  <Gather input="speech dtmf" language="fr-FR"
          action="{PUBLIC_BASE_URL}/bot" method="POST"
          timeout="6" numDigits="1" speechTimeout="auto" />
  <Say language="fr-FR" voice="alice">Je n'ai rien entendu. Au revoir.</Say>
  <Hangup/>
</Response>"""
    return twiml(xml)

@app.post("/bot")
async def bot(request: Request):
    """Reçoit la transcription (SpeechResult) ou les digits (Digits), appelle GPT, et renvoie du TwiML."""
    try:
        form = await request.form()
        speech = (form.get("SpeechResult") or "").strip()
        digits = (form.get("Digits") or "").strip()
        print(f">>> /bot  Speech='{speech}'  Digits='{digits}'", flush=True)

        # Texte utilisateur (parole ou touche)
        if speech:
            user_text = speech
        elif digits:
            user_text = f"Vous avez appuyé sur {digits}."
        else:
            # Rien reçu -> relance Gather
            retry = f"""<?xml version="1.0" encoding="UTF-8"?>
<Response>
  <Say language="fr-FR" voice="alice">Je n'ai pas bien entendu. Pouvez-vous répéter ou appuyer sur une touche ?</Say>
  <Gather input="speech dtmf" language="fr-FR"
          action="{PUBLIC_BASE_URL}/bot" method="POST"
          timeout="6" numDigits="1" />
</Response>"""
            return twiml(retry)

        # 🔧 Appel GPT avec timeout (20s)
        try:
            completion = client_ai_fast.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {
                        "role": "system",
                        "content": (
                            "Tu es un assistant téléphonique sympathique, clair et concis. "
                            "Tu réponds UNIQUEMENT en français, en une ou deux phrases maximum."
                        ),
                    },
                    {"role": "user", "content": user_text}
                ]
            )
            ai_reply = completion.choices[0].message.content.strip()
        except Exception as e:
            print(">>> GPT timeout/err:", repr(e), flush=True)
            ai_reply = "D'accord. Dites autre chose ou appuyez sur une touche."

        # Répondre puis réécouter
        xml = f"""<?xml version="1.0" encoding="UTF-8"?>
<Response>
  <Say language="fr-FR" voice="alice">{ai_reply}</Say>
  <Gather input="speech dtmf" language="fr-FR"
          action="{PUBLIC_BASE_URL}/bot" method="POST"
          timeout="6" numDigits="1" />
</Response>"""
        return twiml(xml)

    except Exception as e:
        print(">>> ERROR in /bot:", repr(e), flush=True)
        fallback = f"""<?xml version="1.0" encoding="UTF-8"?>
<Response>
  <Say language="fr-FR" voice="alice">Oups, petit souci technique. On réessaie.</Say>
  <Gather input="speech dtmf" language="fr-FR"
          action="{PUBLIC_BASE_URL}/bot" method="POST"
          timeout="6" numDigits="1" />
</Response>"""
        return twiml(fallback)

# 🚫 Tue un éventuel ancien serveur sur le port 8000 (utile en notebooks)
os.system("fuser -k 8000/tcp")

# ▶️ Lance le serveur FastAPI en arrière-plan
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

th = threading.Thread(target=run_server, daemon=True)
th.start()
time.sleep(1)
print("✅ Serveur lancé en arrière-plan (FR).")

INFO:     Started server process [3662]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


✅ Serveur lancé en arrière-plan (FR).


🔹 Block 3 – Test du bot directement (sans Twilio)

In [ ]:
import requests

#  URL publique générée par ngrok (Bloc 0)
url = f"{PUBLIC_BASE_URL}/bot"

# On simule ce que Twilio enverrait
data = {"SpeechResult": "Hello, how are you?"}

resp = requests.post(url, data=data)
print("Status:", resp.status_code)
print("Reponse XML:")
print(resp.text)


>>> /bot  Speech='Hello, how are you?'  Digits=''
INFO:     34.148.146.92:0 - "POST /bot HTTP/1.1" 200 OK
Status: 200
Reponse XML:
<?xml version="1.0" encoding="UTF-8"?>
<Response>
  <Say language="fr-FR" voice="alice">Bonjour ! Je suis là pour vous aider. Que puis-je faire pour vous aujourd'hui ?</Say>
  <Gather input="speech dtmf" language="fr-FR"
          action="https://unputridly-roseolar-sharlene.ngrok-free.dev/bot" method="POST"
          timeout="6" numDigits="1" />
</Response>


🔹 Block 4 – Test du serveur local

In [ ]:
import requests, time
time.sleep(0.5)
r = requests.get("http://127.0.0.1:8000/health", timeout=5)
print(r.status_code, r.json())

INFO:     127.0.0.1:37876 - "GET /health HTTP/1.1" 200 OK
200 {'ok': True, 'base': 'https://unputridly-roseolar-sharlene.ngrok-free.dev'}


🔹 Block 5 – Passer un appel Twilio

In [ ]:
import requests

TO_NUMBER = ""  #  mobile validé (ex: 06… → +336…)
resp = requests.post("http://127.0.0.1:8000/call", params={"to": TO_NUMBER}, timeout=20)
print(resp.status_code, resp.text)


>>> /call -> +33652486608
>>> CALL SID = CA0f15d2b950aaebf9c9015cf4e5603441
INFO:     127.0.0.1:42972 - "POST /call?to=%2B33652486608 HTTP/1.1" 200 OK
200 {"sid":"CA0f15d2b950aaebf9c9015cf4e5603441"}
